In [1]:
import pandas as pd
import numpy as np
import os

1. Importar el archivo de la base de datos 'Global shark attack incidents'.

In [2]:
shark_attack = pd.read_csv('GSAF5.csv', encoding = 'latin1')
shark_attack.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


2. Ver el numero de nulos por columna y los nombres de éstas

In [3]:
nulos = shark_attack.isnull().sum()
nulos[nulos > 0]

#tiene un total de 5992 filas y 24 columnas

Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
href formula                 1
href                         3
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

In [4]:
shark_attack.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

3. Eliminar duplicados en base a las columnas con los datos mas representativos, eliminando columnas repetidas o que no aportan ningun tipo de información.

In [5]:
before = len(shark_attack)

select_columns = ['Case Number','Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ']

shark_attack = shark_attack[select_columns].drop_duplicates()

after = len(shark_attack)
print('Numero de registros duplicados: ', str(before - after))


Numero de registros duplicados:  0


In [6]:
shark_attack.columns # Columnas que han quedado

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species '],
      dtype='object')

4. Vuelvo a comprobar los nulos para determinar que columnas son las que pueden aportar mas info. 

In [7]:
nulos = shark_attack.isnull().sum()
nulos

Case Number       0
Date              0
Year              0
Type              0
Country          43
Area            402
Location        496
Activity        527
Name            200
Sex             567
Age            2681
Injury           27
Fatal (Y/N)      19
Time           3213
Species        2934
dtype: int64

 *** Al comprobar los nulos de nuevo, se puede observar que las columnas que estan completas son las referentes a la fecha y el tipo de ataque. Los daños causados y el desenlace del ataque tienen nulos pero no demasiados, por lo que pueden ser interesante.

5. Ordeno las columnas por: Cuando, donde, a quién, como y consecuencias del ataque.

In [8]:
column_order = ['Case Number','Year','Date',
                'Country','Area','Location',
                'Name', 'Sex ','Age',
                'Type','Activity',
                'Injury','Fatal (Y/N)']

shark_attack = shark_attack[column_order]


6. Limpieza de columnas: Year, Case Number, Country, Type, Activity, Injury, Fatal (Y/N)

In [9]:
shark_attack.dtypes # tipo de datos que albergan

Case Number    object
Year            int64
Date           object
Country        object
Area           object
Location       object
Name           object
Sex            object
Age            object
Type           object
Activity       object
Injury         object
Fatal (Y/N)    object
dtype: object

In [10]:
# YEAR:

filteredyear = list(shark_attack[(shark_attack['Year']<=500)].index) 

shark_attack = shark_attack.drop(filteredyear, axis=0)

# Elimino los registros que tienen como "Year" un año menor que 500, ya que son de dudosa veracidad.

In [11]:
#Binning Numeric Variables. Categorizo la columna year y añado columna "Epoca"

bins = [1500,1900,1940,1980,2020]
labels = ["Antes de 1900","1900-1940","1940-1980","1980-Actualidad"]
shark_attack['Epoca'] = pd.cut(shark_attack['Year'], bins=bins, labels=labels)

Epoca_count = shark_attack.groupby(shark_attack.Epoca)['Case Number'].nunique()

In [12]:
# CASE NUMBER--> Month / Columna categorizada Estacion

# Split: dividir fecha y extraer el mes en el que se ha producido el ataque

month = []
for e in shark_attack["Case Number"]:
    aux= e.split(".")
    month.append(aux[1])
    
shark_attack['Month'] = pd.Series(month, index=shark_attack.index) # Añadir columna Month al dataframe


In [13]:
# Observar valores fuera de lo esperado (1-12)

shark_attack.Month.unique() 

filtered1 = shark_attack[(shark_attack['Month'] == '07-10')|(shark_attack['Month'] == '30')] #Filtar erroneos

shark_attack.loc[(shark_attack['Month']=='07-10'),'Month'] = '07'   # Reemplazo de dos registros erroneos
shark_attack.loc[(shark_attack['Month']=='30'),'Month'] = '08'


In [14]:
# Cambiar de string a int y categorizar

shark_attack['Month'] = shark_attack['Month'].astype('int64')  

bins = [-1,0,3,6,9,12]
labels = ['Sin datos','Invierno','Primavera','Verano','Otoño']
shark_attack['Estacion'] = pd.cut(shark_attack['Month'], bins=bins, labels=labels)

Estacion_count = shark_attack.groupby(shark_attack.Estacion)['Case Number'].nunique()

In [15]:
# COUNTRY:

shark_attack = shark_attack.dropna(subset=['Country']) # Elimino registros con un valor nulo

shark_attack['Country'] = shark_attack['Country'].str.upper() # Todo en mayusculas

shark_attack['Country'] = shark_attack['Country'].str.replace('?', '') # Quitar ?
shark_attack['Country'] = shark_attack['Country'].str.strip(' ') # Quitar espacios al principio y fin del string

country_counts = shark_attack['Country'].value_counts() 

attack_country = country_counts.rename_axis('Lugar').reset_index(name='counts')

print (attack_country[0:31]) # Top 30 de paises con mas ataques

               Lugar  counts
0                USA    2099
1          AUSTRALIA    1262
2       SOUTH AFRICA     557
3   PAPUA NEW GUINEA     129
4        NEW ZEALAND     124
5             BRAZIL     102
6            BAHAMAS      95
7             MEXICO      81
8              ITALY      69
9               FIJI      62
10       PHILIPPINES      60
11           REUNION      55
12     NEW CALEDONIA      51
13        MOZAMBIQUE      42
14              CUBA      41
15             SPAIN      39
16             EGYPT      38
17             INDIA      35
18           CROATIA      34
19             JAPAN      32
20            PANAMA      30
21   SOLOMON ISLANDS      27
22              IRAN      25
23         HONG KONG      24
24           JAMAICA      23
25  FRENCH POLYNESIA      21
26         INDONESIA      19
27            GREECE      19
28           ENGLAND      19
29     PACIFIC OCEAN      18
30             TONGA      18


In [16]:
# TYPE --> Esta bastante limpio y sin nulos. 

shark_attack['Type'] = shark_attack['Type'].str.replace('Boating', 'Boat') # Unificar ambos string

Type_count = shark_attack.groupby(shark_attack.Type)['Case Number'].nunique()
Type_count

Type
Boat             307
Invalid          511
Provoked         548
Sea Disaster     208
Unprovoked      4241
Name: Case Number, dtype: int64

In [17]:
# ACTIVITY

activity_counts = shark_attack['Activity'].value_counts() 

attack_activity = activity_counts.rename_axis('Activity').reset_index(name='counts')

# print (attack_activity) # Ver que categorias se repiten mas.

shark_attack['Activity'] = shark_attack['Activity'].fillna("Sin datos")  # Sustituyo los nulos

# Voy a unificar en las siguientes el mayor numero posible: Surfing, Swimming, Fishing, Diving, Bathing, Boating

shark_attack.loc[shark_attack['Activity'].str.contains('surfing'), 'Activity'] = 'Surfing'
shark_attack.loc[shark_attack['Activity'].str.contains('Surf'), 'Activity'] = 'Surfing'
shark_attack.loc[shark_attack['Activity'].str.contains('Surf-'), 'Activity'] = 'Surfing'
shark_attack.loc[shark_attack['Activity'].str.contains('fishing'), 'Activity'] = 'Fishing'
shark_attack.loc[shark_attack['Activity'].str.contains('Fishing '), 'Activity'] = 'Fishing'
shark_attack.loc[shark_attack['Activity'].str.contains('diving'), 'Activity'] = 'Diving'

shark_attack['Activity'] = shark_attack['Activity'].str.replace('Wading', 'Bathing')
shark_attack['Activity'] = shark_attack['Activity'].str.replace('Standing', 'Bathing')
shark_attack['Activity'] = shark_attack['Activity'].str.replace('Snorkeling', 'Bathing')
shark_attack['Activity'] = shark_attack['Activity'].str.replace('Body boarding', 'Surfing')

attack_activity[0:12] # Top 10 actividades mas comunes al ocurrir el ataque. 

# No conseguido unificar el Swimming. Tampoco hay que tener en cuenta el Sin datos, son los nulos.

,Activity,counts
0,Surfing,903
1,Swimming,799
2,Fishing,407
3,Spearfishing,320
4,Bathing,148
5,Wading,141
6,Diving,107
7,Standing,95
8,Snorkeling,76
9,Scuba diving,73


In [18]:
# INJURY -- MUY SUCIA

shark_attack['Injury'] = shark_attack['Injury'].fillna("No details")  # Sustituyo los nulos

# Reunificar un poco los parámetros de la columna

shark_attack.loc[shark_attack['Injury'].str.contains('Lacerations'), 'Injury'] = 'Laceraciones'
shark_attack.loc[shark_attack['Injury'].str.contains('lacerated'), 'Injury'] = 'Laceraciones'
shark_attack.loc[shark_attack['Injury'].str.contains('Minor injury'), 'Injury'] = 'Minor injury'
shark_attack.loc[shark_attack['Injury'].str.contains('No injury'), 'Injury'] = 'No injury'
shark_attack.loc[shark_attack['Injury'].str.contains('bitten'), 'Injury'] = 'Bitten'
shark_attack.loc[shark_attack['Injury'].str.contains('FATAL'), 'Injury'] = 'Mortal'
shark_attack.loc[shark_attack['Injury'].str.contains('Minor injuries'), 'Injury'] = 'Minor injury'

shark_attack.loc[(shark_attack['Injury']!= 'Bitten')&
                 (shark_attack['Injury']!= 'Laceraciones')&
                 (shark_attack['Injury']!= 'Minor injury')&
                 (shark_attack['Injury']!= 'No injury')&
                 (shark_attack['Injury']!= 'Mortal')&
                 (shark_attack['Injury']!= 'No details'), 'Injury'] = 'Varios'

Injury_counts = shark_attack['Injury'].value_counts() 

Injury_counts  # Top 7 de consecuencias del ataque

Varios          1778
Bitten          1326
Mortal           977
Laceraciones     865
No injury        699
Minor injury     115
No details        67
Name: Injury, dtype: int64

In [19]:
# FATAL (Y/N)

shark_attack = shark_attack.rename(columns={'Fatal (Y/N)':'Mortal'})
shark_attack['Mortal'] = shark_attack['Mortal'].fillna("UNKNOWN")  # Sustituyo los nulos

# primero mirar si a parte de Y/N hay otros errores, sustituir

shark_attack['Mortal'] = shark_attack['Mortal'].str.strip(' ') # Quitar espacios al principio y fin del string
shark_attack.loc[(shark_attack['Mortal']=='n'),'Mortal'] = 'UNKNOWN'  
shark_attack.loc[(shark_attack['Mortal']=='F'),'Mortal'] = 'UNKNOWN'
shark_attack.loc[(shark_attack['Mortal']=='#VALUE!'),'Mortal'] = 'UNKNOWN'


shark_attack['Mortal'].value_counts() 

N          4240
Y          1475
UNKNOWN     112
Name: Mortal, dtype: int64

In [20]:
# SEX
shark_attack = shark_attack.rename(columns={'Sex ':'Sex'})

shark_attack['Sex'] = shark_attack['Sex'].fillna("UNKNOWN")  # Sustituyo los nulos

shark_attack.loc[(shark_attack['Sex']!= 'M')&
                 (shark_attack['Sex']!= 'F'), 'Sex'] = 'UNKNOWN'

shark_attack['Sex'].value_counts() 



M          4698
F           570
UNKNOWN     559
Name: Sex, dtype: int64

7. Reordenación de columnas, eliminación de algunas

In [21]:
# Añado una columna nueva como index, eliminando 'Case Number'. Reorganizacion de columnas

shark_attack['Ataques']=shark_attack.index

column_order = ['Ataques','Year','Epoca','Month','Estacion',
                'Country','Sex','Type','Activity','Injury','Mortal']


shark_attack = shark_attack[column_order]

shark_attack.head()

,Ataques,Year,Epoca,Month,Estacion,Country,Sex,Type,Activity,Injury,Mortal
0,0,2016,1980-Actualidad,9,Verano,USA,M,Unprovoked,Surfing,Minor injury,N
1,1,2016,1980-Actualidad,9,Verano,USA,M,Unprovoked,Surfing,Laceraciones,N
2,2,2016,1980-Actualidad,9,Verano,USA,M,Unprovoked,Surfing,Laceraciones,N
3,3,2016,1980-Actualidad,9,Verano,AUSTRALIA,M,Unprovoked,Surfing,Varios,N
4,4,2016,1980-Actualidad,9,Verano,AUSTRALIA,M,Unprovoked,Surfing,No injury,N


In [29]:
# Compruebo los registros nulos. YA no hay ninguno, los he eliminado o los he sustituido por algun parámetro.

nulos = shark_attack.isnull().sum()    
nulos


Ataques     0
Year        0
Epoca       0
Month       0
Estacion    0
Country     0
Sex         0
Type        0
Activity    0
Injury      0
Mortal      0
dtype: int64

In [27]:
# Genero un archivo .csv con el nuevo data set limpio. (5827 rows × 11 columns)

shark_attack.to_csv('shark_attack_limpio.csv', index=False)

## Conclusiones:

- Los meses en los que se producen mas ataques durante julio, agosto y septiembre.
- Los paises en los que hay mas ataques son USA, Australia y Sudáfrica
- Los ataques se producen aparentemente sin relación con la actividad, hay aprox el mismo número de ataques
  registrados mientras hacian surf, nadaban o se bañaban o mientras buceaban.
- En la gran mayoria de los casos no fueron provocados
- Aproximadamente un 25% de los ataques son mortales
- Solo un 10% de los ataques fueron a mujeres

* Me hubiera gustado relacionar los datos de varias columnas para extraer informacion como:
- De los ataques en cada uno de los paises donde se producen mas, que porcentaje son mortales.
- De los ataques en cada uno de los paises donde se producen mas, en que estación hay mayor numero de ataques.
- Relacion entre el tipo de ataque y la actividad que estuviera realizando la victima, y las consecuencias de este.
- etc...